In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
from optimizers import *

Nodes: 21557, Edges: 342353


# GCN results

In [4]:
print("\n=== Before training ===")
test_probs = GCNtest(test_data)
print(f"Sample test predictions (first 5):\n{test_probs[:5]}")

print("\n=== After training ===")
for epoch in range(1, 7):
  loss = GCNtrain()
  test_probs = GCNtest(test_data)
  auc_gcn, f1_gcn, threshold = evaluate_model(test_probs, test_data.edge_label)
  print(f"Epoch {epoch}, Loss: {loss:.4f}, AUC score: {auc_gcn:.4f}, F1 score: {f1_gcn:.4f}")
  print(f"Sample predictions: {test_probs[:5].tolist()}...")



=== Before training ===
Sample test predictions (first 5):
[0.52669513 0.5079812  0.5114555  0.5060205  0.5151084 ]

=== After training ===
Epoch 1, Loss: 1.3794, AUC score: 0.7138, F1 score: 0.7117
Sample predictions: [0.6533562541007996, 0.5742816925048828, 0.5738202333450317, 0.5693365931510925, 0.585483729839325]...
Epoch 2, Loss: 1.4429, AUC score: 0.7730, F1 score: 0.7248
Sample predictions: [0.5268257856369019, 0.5107012987136841, 0.5146665573120117, 0.5169265270233154, 0.5147409439086914]...
Epoch 3, Loss: 1.3598, AUC score: 0.7745, F1 score: 0.7382
Sample predictions: [0.7236695289611816, 0.6123666167259216, 0.6401826739311218, 0.6087967753410339, 0.618206262588501]...
Epoch 4, Loss: 1.4717, AUC score: 0.7757, F1 score: 0.7380
Sample predictions: [0.6059471964836121, 0.5497739315032959, 0.5638862252235413, 0.5500574707984924, 0.5542542934417725]...
Epoch 5, Loss: 1.3474, AUC score: 0.8039, F1 score: 0.7464
Sample predictions: [0.5413625240325928, 0.5190430283546448, 0.5263428

# GAN results

In [4]:
print("\n=== Before training ===")
test_probs = GANtest(test_data)
print(f"Sample test predictions (first 5):\n{test_probs[:5]}")

print("\n=== After training ===")
for epoch in range(1, 7):
    d_loss, g_loss = GANtrain()
    test_probs = GANtest(test_data)
    auc_gan, f1_gan, threshold = evaluate_model(test_probs, test_data.edge_label)
    print(f"Epoch {epoch}, D Loss: {d_loss:.4f}, G Loss: {g_loss:.4f}, AUC score: {auc_gan:.4f}, F1 score: {f1_gan:.4f}")
    print(f"Sample predictions: {test_probs[:5].tolist()}...")



=== Before training ===
Sample test predictions (first 5):
[[0.5500396 ]
 [0.4581731 ]
 [0.5114905 ]
 [0.46558258]
 [0.4642037 ]]

=== After training ===
Epoch 1, D Loss: 0.7785, G Loss: 0.0006, AUC score: 0.5053, F1 score: 0.6667
Sample predictions: [[0.5472835302352905], [0.4675036668777466], [0.5151628851890564], [0.47208327054977417], [0.4706752598285675]]...
Epoch 2, D Loss: 0.4838, G Loss: -0.0165, AUC score: 0.5058, F1 score: 0.6667
Sample predictions: [[0.5446469783782959], [0.47500020265579224], [0.5178864598274231], [0.476769357919693], [0.47607240080833435]]...
Epoch 3, D Loss: 0.3200, G Loss: -0.0295, AUC score: 0.5063, F1 score: 0.6667
Sample predictions: [[0.5436295866966248], [0.4795844256877899], [0.5203239321708679], [0.48019737005233765], [0.47955191135406494]]...
Epoch 4, D Loss: 0.2235, G Loss: -0.0416, AUC score: 0.5071, F1 score: 0.6667
Sample predictions: [[0.5433065295219421], [0.48325660824775696], [0.5223309397697449], [0.48367345333099365], [0.48208966851234

# Results after optimization with Genetic Algorithm

## GCN

In [12]:
result = run_gcn_ga()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 35
  Learning Rate   : 0.01466
  Num Layers      : 3
  Dropout         : 0.21

Performance Metrics:
  F1 Score        : 0.8471
  AUC             : 0.9044
  Average Loss    : 1.2605


## GAN

In [ ]:
result = run_gan_ga()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")

# Results after optimization with Particle Swarm Optimization

## GCN

In [5]:
result = run_gcn_pso()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 32
  Learning Rate   : 0.01579
  Num Layers      : 4
  Dropout         : 0.00

Performance Metrics:
  F1 Score        : 0.8498
  AUC             : 0.8287
  Average Loss    : 1.3822


# Results after optimization with Simulated Annealing

## GCN

In [9]:
result = run_gcn_sa()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 112
  Learning Rate   : 0.03100
  Num Layers      : 3
  Dropout         : 0.55

Performance Metrics:
  F1 Score        : 0.8393
  AUC             : 0.8354
  Average Loss    : 1.3810


# Results after optimization with Artificial Bee Colony

## GCN

In [12]:
result = run_gcn_abc()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 32
  Learning Rate   : 0.02367
  Num Layers      : 3
  Dropout         : 0.48

Performance Metrics:
  F1 Score        : 0.8490
  AUC             : 0.5000
  Average Loss    : 31.2235


# Results after optimization with Hill Climbing

## GCN

In [6]:
result = run_gcn_hc()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 58
  Learning Rate   : 0.02508
  Num Layers      : 3
  Dropout         : 0.54

Performance Metrics:
  F1 Score        : 0.8491
  AUC             : 0.7861
  Average Loss    : 1.3760


# Results after optimization with Random Search

## GCN

In [7]:
result = run_gcn_ra()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 125
  Learning Rate   : 0.01720
  Num Layers      : 3
  Dropout         : 0.50

Performance Metrics:
  F1 Score        : 0.8411
  AUC             : 0.9035
  Average Loss    : 1.2390
